# AutoGen 基本範例

在這個程式碼範例中，您將使用 [AutoGen](https://aka.ms/ai-agents/autogen) AI 框架來建立一個基本代理。

此範例的目的是向您展示我們稍後在其他程式碼範例中實現不同代理模式時將使用的步驟。


## 匯入所需的 Python 套件


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## 建立客戶端

在此範例中，我們將使用 [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) 來存取 LLM。

`model` 被定義為 `gpt-4o-mini`。嘗試將模型更改為 GitHub Models 市場中提供的其他模型，看看不同的結果。

作為快速測試，我們將執行一個簡單的提示 - `法國的首都是什麼？`


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## 定義代理

現在我們已經設置了 `client` 並確認其正常運作，接下來讓我們創建一個 `AssistantAgent`。每個代理可以分配以下屬性：
**name** - 一個簡短的名稱，方便在多代理流程中引用。
**model_client** - 你在前一步中創建的客戶端。
**tools** - 代理可以使用的工具，用於完成任務。
**system_message** - 定義任務、行為和語氣的元提示。

你可以更改 system message 來觀察 LLM 的回應。我們將在第4課中討論 `tools`。


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## 執行代理

以下函數將執行代理。我們使用 `on_message` 方法來更新代理的狀態，加入新的訊息。

在此情況下，我們使用來自使用者的新訊息更新狀態，訊息內容為「幫我規劃一個陽光明媚的完美假期」。

您可以更改訊息內容，看看大型語言模型（LLM）如何做出不同的回應。


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**免責聲明**：  
本文件使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。應以原始語言的文件作為權威來源。對於關鍵資訊，建議尋求專業人工翻譯。我們對因使用此翻譯而產生的任何誤解或錯誤解讀概不負責。
